<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 10


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс Invoice в C#, который будет представлять информацию о
фактурах за поставленные товары или оказанные услуги. На основе этого класса
разработать 2-3 производных класса, демонстрирующих принципы наследования и
полиморфизма. В каждом из классов должны быть реализованы новые атрибуты и
методы, а также переопределены некоторые методы базового класса для
демонстрации полиморфизма.

Требования к базовому классу Invoice:

• Атрибуты: Номер фактуры (InvoiceNumber), Дата выдачи (IssueDate), Общая
сумма (TotalAmount).

• Методы:
1. CalculateTotal(): метод для расчета общей суммы по фактуре.
2. AddLine(LineItem lineItem): метод для добавления позиции в фактуру.
3. RemoveLine(LineItem lineItem): метод для удаления позиции из
фактуры.

Требования к производным классам:
1. ТоварнаяФактура (GoodsInvoice): Должна содержать дополнительные
атрибуты, такие как Дата поставки (SupplyDate). Метод AddLine() должен
быть переопределен для добавления информации о дате поставки товара
при добавлении позиции.
2. УслуговаяФактура (ServiceInvoice): Должна содержать дополнительные
атрибуты, такие как Дата оказания услуги (ServiceDate).
Метод RemoveLine() должен быть переопределен для добавления
информации о причине аннулирования услуги при удалении позиции.
3. КомбинированнаяФактура (CombinedInvoice) (если требуется третий класс):
Должна содержать дополнительные атрибуты, такие как Наличие возврата
(ReturnAllowed). Метод CalculateTotal() должен быть переопределен для
учета возможного возврата товара или услуги при расчете общей суммы.

#### Дополнительное задание
Добавьте к сущестующим классам конструктора классов с использованием гетторов и сетторов и реализуйте взаимодействие объектов между собой

<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [1]:
using System;
using System.Collections.Generic;

public class LineItem
{
    public string Description { get; set; }
    public decimal Price { get; set; }
    public int Quantity { get; set; }

    public LineItem(string description, decimal price, int quantity)
    {
        Description = description;
        Price = price;
        Quantity = quantity;
    }

    public decimal GetTotal()
    {
        return Price * Quantity;
    }
}

abstract class Invoice
{
    public string Description { get; set; }
    public string InvoiceNumber { get; set; }
    public DateTime IssueDate { get; set; }
    public decimal TotalAmount { get; protected set; }

    protected List<LineItem> lineItems;

    public Invoice(string invoiceNumber, DateTime issueDate)
    {
        InvoiceNumber = invoiceNumber;
        IssueDate = issueDate;
        lineItems = new List<LineItem>();
    }

    public virtual void CalculateTotal()
    {
        TotalAmount = 0;
        foreach (var item in lineItems)
        {
            TotalAmount += item.GetTotal();
        }
    }

    public virtual void AddLine(LineItem lineItem)
    {
        lineItems.Add(lineItem);
    }

    public virtual void RemoveLine(LineItem lineItem)
    {
        lineItems.Remove(lineItem);
    }

    public List<LineItem> GetLineItems()
    {
        return lineItems;
    }
}

// Товарная фактура
class GoodsInvoice : Invoice
{
    public DateTime SupplyDate { get; set; }

    public GoodsInvoice(string invoiceNumber, DateTime issueDate, DateTime supplyDate)
        : base(invoiceNumber, issueDate)
    {
        SupplyDate = supplyDate;
    }

    public override void AddLine(LineItem lineItem)
    {
        base.AddLine(lineItem);
        Console.WriteLine($"Товар добавлен с датой поставки: {SupplyDate.ToShortDateString()}");
    }
}

// Услуговая фактура
class ServiceInvoice : Invoice
{
    public DateTime ServiceDate { get; set; }

    public ServiceInvoice(string invoiceNumber, DateTime issueDate, DateTime serviceDate)
        : base(invoiceNumber, issueDate)
    {
        ServiceDate = serviceDate;
    }

    public override void AddLine(LineItem lineItem)
    {
        base.AddLine(lineItem);
        Console.WriteLine($"Дата оказания услуги: {ServiceDate.ToShortDateString()}");
    }

    public override void RemoveLine(LineItem lineItem)
    {
        base.RemoveLine(lineItem);
        Console.WriteLine($"Услуга: {lineItem.Description} была аннулирована.\nПричина: ошибка заказа.");
    }
}

// Комбинированная фактура
class CombinedInvoice : Invoice
{
    public bool ReturnAllowed { get; set; }

    public CombinedInvoice(string invoiceNumber, DateTime issueDate, bool returnAllowed)
        : base(invoiceNumber, issueDate)
    {
        ReturnAllowed = returnAllowed;
    }

    public override void CalculateTotal()
    {
        base.CalculateTotal();

        if (ReturnAllowed) 
        {
            int totalItemsCount = 0; 
            decimal totalDiscount = 0;

            foreach (var item in lineItems)
            {
                int itemQuantity = item.Quantity;
                totalItemsCount += itemQuantity; 

                for (int i = 0; i < itemQuantity; i++)
                {
                    totalDiscount += 50;
                    Console.WriteLine($"Возврат разрешен на товар: {item.Description}. Применена скидка 50.");
                }
            }

            TotalAmount -= totalDiscount; 
            Console.WriteLine($"Общая скидка: {totalDiscount} на {totalItemsCount} товаров.");
        }
    }

    public void AddInvoiceItems(Invoice invoice)
    {
        foreach (LineItem item in invoice.GetLineItems())
        {
            this.AddLine(item);
        }
    }
}

// Создание товарной фактуры
GoodsInvoice goodsInvoice = new GoodsInvoice("INV001", DateTime.Now, DateTime.Now.AddDays(7));
goodsInvoice.AddLine(new LineItem("Товар 1", 100, 3));
goodsInvoice.AddLine(new LineItem("Товар 2", 200, 1));
goodsInvoice.CalculateTotal();
Console.WriteLine($"Общая сумма по товарной фактуре: {goodsInvoice.TotalAmount}");

Console.WriteLine("\n\n\n\n");

// Создание услуговой фактуры
ServiceInvoice serviceInvoice = new ServiceInvoice("INV002", DateTime.Now, DateTime.Now);
serviceInvoice.AddLine(new LineItem("Услуга 1", 300, 1));
serviceInvoice.RemoveLine(new LineItem("Услуга 2", 400, 1));
serviceInvoice.CalculateTotal();
Console.WriteLine($"Общая сумма по услуговой фактуре: {serviceInvoice.TotalAmount}");

Console.WriteLine("\n\n\n\n");

// Создание комбинированной фактуры и добавление позиций из других фактур
CombinedInvoice combinedInvoice = new CombinedInvoice("INV003", DateTime.Now, true);
combinedInvoice.AddInvoiceItems(goodsInvoice);
combinedInvoice.AddInvoiceItems(serviceInvoice);
combinedInvoice.CalculateTotal();
Console.WriteLine($"Общая сумма по комбинированной фактуре: {combinedInvoice.TotalAmount}");


Товар добавлен с датой поставки: 20.09.2024
Товар добавлен с датой поставки: 20.09.2024
Общая сумма по товарной фактуре: 500





Дата оказания услуги: 13.09.2024
Услуга: Услуга 2 была аннулирована.
Причина: ошибка заказа.
Общая сумма по услуговой фактуре: 300





Возврат разрешен на товар: Товар 1. Применена скидка 50.
Возврат разрешен на товар: Товар 1. Применена скидка 50.
Возврат разрешен на товар: Товар 1. Применена скидка 50.
Возврат разрешен на товар: Товар 2. Применена скидка 50.
Возврат разрешен на товар: Услуга 1. Применена скидка 50.
Общая скидка: 250 на 5 товаров.
Общая сумма по комбинированной фактуре: 550
